In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error

In [7]:
m = pd.read_csv("../../data/processed/KOBIS_개봉일람_3.csv")

In [8]:
m

,감독,제작사,배급사,전국 스크린수,전국 관객수,국적_기타,국적_독일,국적_러시아,국적_미국,국적_스페인,...,장르_어드벤처,장르_전쟁,장르_코미디,장르_판타지,등급_12세이상관람가,등급_15세이상관람가,등급_전체관람가,등급_청소년관람불가,영화구분_독립/예술영화,영화구분_일반영화
0,14.306190,12.886638,13.607103,1587,17613682,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1,15.172542,16.604523,13.607103,1978,16264944,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
2,15.714698,16.402393,12.960970,1912,14410754,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,True
3,15.908342,15.263750,13.607103,966,14245998,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,14.740589,15.240150,13.427532,2835,13934592,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324,7.217443,7.217443,9.570018,82,1362,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
2325,7.205635,7.205635,7.205635,60,1346,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2326,7.156956,8.649415,9.092294,71,1282,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
2327,7.770353,7.134891,7.134891,79,1254,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [14]:
# 변수 제거 함수
def recursive_feature_elimination(X, y, model, min_features=1, verbose=True):
    # 초기 변수 개수
    n_features = X.shape[1]
    
    # 현재 변수들의 성능
    best_score = float("inf")
    best_features = X.columns.tolist()
    
    # 최소 변수 개수 이상인 경우
    while n_features > min_features:
        # 모든 변수에 대해 반복
        scores = []
        for feature in X.columns:
            # 선택한 변수 제외
            features = X.columns.drop(feature)
            X_new = X[features]
            
            # 모델 학습 및 평가
            model.fit(X_new, y)
            y_pred = model.predict(X_new)
            score = mean_squared_error(y, y_pred)
            scores.append(score)
            
        # 가장 성능이 좋은 변수 선택
        idx = pd.Index(scores).argmin()
        worst_feature = X.columns[idx]
        
        # 변수 제거
        X.drop(worst_feature, axis=1, inplace=True)
        n_features = X.shape[1]
        if verbose:
            print(f"Removing {worst_feature}: {n_features} features left")
        
        # 현재 변수 개수에서의 성능이 최선인 경우
        if min(scores) < best_score:
            best_score = min(scores)
            best_features = X.columns.tolist()
        else:
            break
    
    # 최종 선택된 변수들
    return best_features

# ElasticNetCV 모델 객체 생성
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
enet = ElasticNetCV(alphas=alphas, l1_ratio=0.5, cv=5)

# 최소 변수 개수 지정
min_features = 5

# 재귀적 변수 제거 수행
X = m.drop("전국 관객수", axis=1)
y = m["전국 관객수"]
selected_features = recursive_feature_elimination(X, y, enet, min_features=min_features)

# 선택된 변수 출력
print(f"{len(selected_features)} features selected:")
print(selected_features)

c:\Users\lbw\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29518004093902.953, tolerance: 16523375746.153503
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\lbw\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1733018755870277.8, tolerance: 804146052204.6497
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\lbw\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1930413845179179.5, tolerance: 779389215051.8561
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\lbw\anaconda3\envs\DACON-AI\Lib\site-packages

39 features selected:
['감독', '제작사', '배급사', '전국 스크린수', '국적_독일', '국적_러시아', '국적_미국', '국적_스페인', '국적_영국', '국적_일본', '국적_중국', '국적_한국', '국적_홍콩', '장르_SF', '장르_가족', '장르_공연', '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스', '장르_뮤지컬', '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)', '장르_스릴러', '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지', '등급_12세이상관람가', '등급_15세이상관람가', '등급_전체관람가', '등급_청소년관람불가', '영화구분_독립/예술영화', '영화구분_일반영화']


c:\Users\lbw\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1699273091891437.0, tolerance: 767063953431.2975
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\lbw\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1133953337126454.5, tolerance: 763770152197.1346
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\lbw\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3435098468310.25, tolerance: 16523375746.153503
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\lbw\anaconda3\envs\DACON-AI\Lib\site-packages\s

In [12]:
# 데이터 분할
X = m[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)